In [ ]:
# import the required lib
import datetime
import re
import time

'''
To get query condition string from reading the file :singleDBCondition
and handle it to gain all parameters by split function
'''
DB_CONN_ID = ''
END_TIME = ''
INTERVAL = 0
START_TIME = ''
REPORT_TYPE = ''
ISOK_ALL_PARA = 1
DBs = []
GENERAL_ERROR = None

def handle_query_condition():
    # To declare below variables are global variables
    global DB_CONN_ID
    global END_TIME
    global INTERVAL
    global START_TIME
    global REPORT_TYPE
    global ISOK_ALL_PARA
    global DBs
    global GENERAL_ERROR
    
    # To read parameters from singleDBCondition file
    where_cause = !cat enterpriseCondition
    where_cause = re.search(r'DB_CONN_ID\s?=\s?.+END_TIME\s?=\s?.+INTERVAL\s?=\s?.+REPORT_TYPE\s?=\s?.+', where_cause[0])
    GENERAL_ERROR = []
    if (where_cause <> None):
        all_vars = where_cause.group().split(' ')
        if len(all_vars) <> 4:
            err_str = 'The format of parameters is error.'
            if err_str not in GENERAL_ERROR:
                GENERAL_ERROR.append(err_str)
            ISOK_ALL_PARA = 0
        else:# len(all_vars) == 4
            # To get DB_CONN_ID
            if (re.search(r'DB_CONN_ID\s?=\s?(.+)', all_vars[0]) == None):
                err_str = 'The DB_CONN_ID can not be empty.'
                if err_str not in GENERAL_ERROR:
                    GENERAL_ERROR.append(err_str)
                ISOK_ALL_PARA = 0
            else:
                DB_CONN_ID = re.search(r'DB_CONN_ID\s?=\s?(.+)', all_vars[0]).group(1)
                if (DB_CONN_ID <> '*'):
                    DBs = DB_CONN_ID.split(',')
                else:# * stands for getting all connection name from repository db
                    dataframe_dbs = %sql select distinct substr(dbconn_id,1,30) as conn_name from ibm_dsm_views.throughput_all
                    dbs = list(dataframe_dbs['CONN_NAME'].values)
                    DBs = []
                    for id_db in range(len(dbs)):
                        DBs.append(str(dbs[id_db]).strip())

            # To get the value of INTERVAL
            if (re.search(r'INTERVAL\s?=\s?([0-9]+$)', all_vars[2]) == None):
                err_str = 'The format of INTERVAL error.'
                if err_str not in GENERAL_ERROR:
                    GENERAL_ERROR.append(err_str)
                ISOK_ALL_PARA = 0
            else:
                INTERVAL = int(re.search(r'(\d+)', all_vars[2]).group())
                if (INTERVAL > 100):  # most get 100 data
                    INTERVAL = 100

            # To get START_TIME according to END_TIME
            if (re.search(r'(\d{4}-\d{1,2}-\d{1,2})', all_vars[1]) == None) | ( re.search(r'(\d{1,2}:\d{1,2}:\d{1,2})', all_vars[1]) == None):
                err_str = 'The format of END_TIME error.'
                if err_str not in GENERAL_ERROR:
                    GENERAL_ERROR.append(err_str)
                
                ISOK_ALL_PARA = 0
            else:
                END_TIME = re.search(r'(\d{4}-\d{1,2}-\d{1,2})', all_vars[1]).group() + ' ' + re.search( r'(\d{1,2}:\d{1,2}:\d{1,2})', all_vars[1]).group()
                def is_valid_datetime(END_TIME):
                    try:
                        time.strptime(END_TIME, '%Y-%m-%d %H:%M:%S')
                        return True
                    except:
                        return False
                
                if is_valid_datetime(END_TIME) == True:
                    datetime_tuple = time.strptime(END_TIME, '%Y-%m-%d %H:%M:%S')
                    # To slice datetime_tuple to gain exact time data
                    year, month, day, hour, minute, second = datetime_tuple[:6]
                    final_time = datetime.datetime(year, month, day, hour, minute, second) + datetime.timedelta(hours = -INTERVAL)
                    # To transfer the datetime fields to string
                    START_TIME = final_time.strftime('%Y-%m-%d %H:%M:%S')
                else:
                    err_str = 'The format of END_TIME error.'
                    if err_str not in GENERAL_ERROR:
                        GENERAL_ERROR.append(err_str)
                    ISOK_ALL_PARA = 0
            # To get the value of REPORT_TYPE
            if (re.search(r'REPORT_TYPE\s?=\s?(.+)', all_vars[3]) == None):
                err_str = 'The format of REPORT_TYPE error.'
                if err_str not in GENERAL_ERROR:
                    GENERAL_ERROR.append(err_str)
                ISOK_ALL_PARA = 0
            else:
                REPORT_TYPE = re.search(r'REPORT_TYPE\s?=\s?(.+)', all_vars[3]).group(1).upper()
                if (REPORT_TYPE != 'ALL' and REPORT_TYPE != 'RESOURCE' and REPORT_TYPE != 'CPU' and REPORT_TYPE != 'MEMORY' and REPORT_TYPE != 'IO' and REPORT_TYPE != 'LOG'):
                    err_str = 'The format of REPORT_TYPE error.'
                    if err_str not in GENERAL_ERROR:
                        GENERAL_ERROR.append(err_str)
                    ISOK_ALL_PARA = 0

'''
This function is to get all datatime string according to the result 
queried from database.

The variable ori_datetime_str is a dictionary and its data 
from the combination of hour_list and date_all, which will 
be used for judging whether some data exists in it or not.
'''
def get_original_datatime_str(date_all, hour_list):
    ori_datetime_str = {}
    for indx in range(len(hour_list)):
        tmp_date_str = date_all[indx].encode('unicode-escape').decode('string_escape')
        tmp_hour_str = str(hour_list[indx])
        tmp_datetime_str = ''
        if(len(tmp_hour_str) == 1):#Change 1:00:00 into 01:00:00
            tmp_hour_str = '0' + tmp_hour_str
        tmp_datetime_str = tmp_date_str + ' ' + tmp_hour_str + ':00:00'
        ori_datetime_str[tmp_datetime_str] = indx
    return ori_datetime_str

#To store previous date string value as a reference data
def format_x_axis(date_all, hour_list, x_ticks, x_ticks_lables):
    pre_date_str = str(date_all[0])

    for dateIdx in range(len(date_all)):
        # To get the data for the x-axis ticks
        x_ticks.append(float('%0.1f' % dateIdx))
        '''
        Get the data for the lable of x-axis
        If the label existed in the list x_ticks_lables,
        put hour_str into x_ticks_lables 
        otherwise,put date_lables into x_ticks_lables
        '''
        hour_str = str(hour_list[dateIdx])
        if len(hour_str) == 1:
            hour_str = '0' + hour_str
        date_str = str(date_all[dateIdx])
        x_lables = date_str + ' ' + hour_str
        if (dateIdx == 0):
            x_ticks_lables.append(x_lables)
        else:#dateIdx > 0
            if (pre_date_str == date_str):
                x_ticks_lables.append(hour_str)
            else:
                pre_date_str = date_str
                x_ticks_lables.append(x_lables)

In [ ]:
import numpy as np
import pandas as pd
import os

handle_query_condition()

#To print the error of query conditon, only print once
for err_id in range(len(GENERAL_ERROR)):
    print GENERAL_ERROR[err_id]
    
resource_df_empty = False

db_str = ""
for id_1 in range(len(DBs)):
    if id_1 == len(DBs) - 1:
        db_str += "'" + DBs[id_1] + "'"
    else:
        db_str += "'" + DBs[id_1] + "'" + ","
        
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'RESOURCE')):
    #dataframe_week = %sql select year(collected) as year, week(collected) as week, substr(dbconn_id,1,30) as conn_name, date(min(collected)) as begin_date, date(max(collected)) as end_date, sum(total_cpu_usec_delta) / 1000000.0 as cpu_sec, sum(logical_reads_delta) as logical_reads, sum(physical_reads_delta) as physical_reads, sum(act_completed_total_delta) as activities, sum(total_app_commits_delta) as commits, sum(total_act_time_delta) / (sum(act_aborted_total_delta) + sum(act_completed_total_delta)) as avg_activity_time_msec from ibm_dsm_views.throughput_all group by year(collected), week(collected), dbconn_id order by year(collected), week(collected), dbconn_id
    dataframe_hour = %sql select char(date(collected)) as date, hour(collected) as hours, substr(dbconn_id,1,30) as conn_name, date(min(collected)) as begin_date, date(max(collected)) as end_date, sum(total_cpu_usec_delta) / 1000000.0 as cpu_sec, sum(logical_reads_delta) as logical_reads, sum(physical_reads_delta) as physical_reads, sum(act_completed_total_delta) as activities, sum(total_app_commits_delta) as commits, sum(total_act_time_delta) / (sum(act_aborted_total_delta) + sum(act_completed_total_delta)) as avg_activity_time_msec from ibm_dsm_views.throughput_all where  dbconn_id in ({db_str}) and collected >= '{START_TIME}' and collected < '{END_TIME}' group by date(collected), hour(collected), dbconn_id order by date, hours, dbconn_id
    if os.path.exists("resource.csv"):
        !rm resource.csv
    if dataframe_hour.empty:
        resource_df_empty = True
        print 'For resource: The query result is empty, please check your query parameters.\n'
    else:
        #To save log data for generating table
        dataframe_hour.to_csv("resource.csv", index_label = "INDEX")

In [ ]:
#import the required lib
import numpy as np
import pylab as pl
from scipy.interpolate import spline

#To get all query conditions
handle_query_condition()
        
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'RESOURCE')):
    all_logical_read_list = []
    x_ticks = []
    x_ticks_lables = []
    y_logical_reads_max = 0
    legend_title_logical = []
    log_mil = False
    for id_1 in range(len(DBs)):
        #Get data from the target database to assemble a data frame that will be used for the following graphs
        dataframe_hour = %sql select char(date(collected)) as date, hour(collected) as hours, sum(total_cpu_usec_delta) / 1000000.0 as cpu_sec, sum(logical_reads_delta) as logical_reads, sum(PHYSICAL_READS_DELTA) as physical_reads, sum(total_act_time)/(sum(act_aborted_total)+sum(act_completed_total)) as avg_activity_time_msec,sum(act_completed_total_delta) as activities from ibm_dsm_views.throughput_all where dbconn_id = '{DBs[id_1]}' and collected >= '{START_TIME}' and collected < '{END_TIME}' group by date(collected), hour(collected) order by date, hours
        if dataframe_hour.empty:
            if resource_df_empty == False:
                print 'For resource(Logical Reads): The query result is empty for the database connection ' + DBs[id_1] + ', please check your query parameters.\n'
        else:
            legend_title_logical.append(DBs[id_1])
            #Get the hour data as x-axis from the data frame
            x_hour = dataframe_hour['HOURS']
            y_logical_reads = dataframe_hour['LOGICAL_READS']
            y_logical_reads_list = list(y_logical_reads)
            if (y_logical_reads_max < y_logical_reads.max()):
                y_logical_reads_max = y_logical_reads.max()

            x_hour_list = list(x_hour) #here x_hour is a python list
            date_all = list(dataframe_hour['DATE'].values)

            ori_datetime_str = get_original_datatime_str(date_all, x_hour_list)

            '''
            When the data queried is not equal to the requirement. Filling missing data 
            into the list x_hour_list,date_all and y_ticks.
            '''
            if (len(x_hour_list) < INTERVAL):
                y_logical_reads_list = []
                #Transfer datatime data into time tuple for getting its' timestamp
                min_dt_str = START_TIME[0:13] + ':00:00' 
                tm_tuple = time.strptime(min_dt_str, '%Y-%m-%d %H:00:00')
                min_timestamp = time.mktime(tm_tuple)
            
                date_ref_str = END_TIME[0:13] + ':00:00'
                tmp_tuple = time.strptime(date_ref_str, '%Y-%m-%d %H:00:00')
                max_timestamp = time.mktime(tmp_tuple)
                '''
                Get the difference between the max_timestamp and min_timestamp 
                which will be used for gaining all date and hour including the missing
                '''
                hour_diff = int((max_timestamp - min_timestamp) / 3600)
                #clear date_all
                date_all = []
                #clear x_hour_list
                x_hour_list = []
                #clear y_ticks
                
                #Reassign three var above
                for id_2 in range(hour_diff):
                    tmp_st = min_timestamp + id_2 * 3600
                    #Trans timestamp to datetime string
                    tmp_datetime = datetime.datetime.fromtimestamp(tmp_st)
                    tmp_datetime_str = tmp_datetime.strftime("%Y-%m-%d %H:%M:%S")
                    tmp_date_str = tmp_datetime_str[0:10]
                    tmp_hour_str = tmp_datetime_str[11:13]
                    date_all.append(tmp_date_str)
                    x_hour_list.append(tmp_hour_str)
                    
                    '''
                    Below code is for handling y-axis's data
                    If there is no data at this hour,0.0 will be filled into
                    '''
                    if tmp_datetime_str in ori_datetime_str:
                        tmp_index = ori_datetime_str[tmp_datetime_str]
                        y_logical_reads_list.append(list(y_logical_reads)[tmp_index])
                    else:
                        y_logical_reads_list.append(0)
            all_logical_read_list.append(y_logical_reads_list)
           
            #previous date string value as a reference data
            pre_date_str = str(date_all[0])
            
            if len(x_ticks) == 0:
                for id_3 in range(len(date_all)):
                    #Get the data for the x-axis ticks
                    x_ticks.append(float('%.1f' % id_3))

                    '''
                    Get the data for the lable of x-axis
                    If the label existed in the list x_ticks_lables,
                    put hour_str into x_ticks_lables 
                    otherwise,put date_lables into x_ticks_lables
                    '''
                    hour_str = str(x_hour_list[id_3])
                    date_str = str(date_all[id_3])
                    x_lables = date_str + ' ' + hour_str
                    if (id_3 == 0):
                        x_ticks_lables.append(x_lables)
                    else:#id_3 > 0
                        if (pre_date_str == date_str):
                            x_ticks_lables.append(hour_str)
                        else:
                            pre_date_str = date_str
                            x_ticks_lables.append(x_lables)
                        
    if len(all_logical_read_list) > 0:
        #Declare a Sketchpad as the first graph
        fig = pl.figure()
        ax_log = fig.add_subplot(111)
        #Declare an ax container as the first drawing paper
        x_lable= 'Hours'
        data_size = len(x_ticks)
        if(data_size <= 20):
            fig.set_size_inches(12,6)
        elif(data_size <= 40):
            fig.set_size_inches(16,6)
        elif(data_size <= 60):
            fig.set_size_inches(18,7)
        elif(data_size <=100):
            fig.set_size_inches(22,7)

        pl.xlabel(x_lable, fontsize = 12)
        #Set title for the second graph
        figure_title='Logical Reads by Hour\n'
        pl.title(figure_title, fontsize = 14, fontweight = 'bold')
        pl.xticks(x_ticks, x_ticks_lables, rotation = 90)
        #To set grid line style according to your requirement
        pl.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
        
        #Set y-axis label
        if y_logical_reads.max() > 1000000:
            log_mil = True
            for id_4 in range(len(all_logical_read_list)):
                for id_4_0 in range(len(all_logical_read_list[id_4])):
                    all_logical_read_list[id_4][id_4_0] = all_logical_read_list[id_4][id_4_0]/1000000.0
        
        xnew_hour = []
        ynew_logical_reads = []
        for id_5 in range(len(all_logical_read_list)):
            #To mark the data point 
            for id_6 in range(len(all_logical_read_list[id_5])):
                if(all_logical_read_list[id_5][id_6] == 0.0):#If no data,drawing a empty circle
                    pl.scatter(x_ticks[id_6],all_logical_read_list[id_5][id_6], c = '', marker = 'o', edgecolors = 'r', s = 50)
                else:
                    pl.scatter(x_ticks[id_6], all_logical_read_list[id_5][id_6], c = '#2c628b') 
                    pl.text(x_ticks[id_6], all_logical_read_list[id_5][id_6], '%.1f' % all_logical_read_list[id_5][id_6], fontsize = 9)
        
            if(len(all_logical_read_list[id_5]) >= 3):
                ##In order to smooth the line chart,handle the data further##
                #Expand each x axis data 20 times
                xnew_hour = np.linspace(np.asarray(x_ticks).min(), np.asarray(x_ticks).max(), np.asarray(x_ticks).size*20) 
                #Handle the data of new y axis data
                ynew_logical_reads = spline(np.asarray(x_ticks), all_logical_read_list[id_5], xnew_hour)
                ynew_logical_reads_list = list(ynew_logical_reads)
                #No negative value for y-axis
                for id_7 in range(len(ynew_logical_reads_list)):
                    if (ynew_logical_reads_list[id_7] < 0.0):
                        ynew_logical_reads_list[id_7] = 0.0
                ynew_logical_reads = np.asarray(ynew_logical_reads_list)        
                #Fill the gragh according to your requirement
                #pl.fill_between(xnew_hour, ynew_logical_reads, where=(xnew_hour.min()<xnew_hour) & (xnew_hour<xnew_hour.max()), color = '#2c628b', alpha = 0.09)
                #Draw curve graph
                pl.plot(xnew_hour, ynew_logical_reads)
               
        if (log_mil == True):
            pl.ylabel(u'LOGICAL_READS(mil times)', fontsize = 12)
        else:
            pl.ylabel(u'LOGICAL_READS(times)',fontsize=12)
        #pl.legend(labels = ['a', 'b'], loc = 'best')
        #Set the legends for the both graphs
        box_log = ax_log.get_position()
        ax_log.set_position([box_log.x0, box_log.y0 + box_log.height * 0.1, box_log.width, box_log.height * 0.9])
        ax_log.legend(legend_title_logical, fontsize = 11, loc = 'upper center', bbox_to_anchor=(0.5,1.08), fancybox = True, shadow = True, ncol = len(legend_title_logical))
        pl.show()

In [ ]:
#import the required lib
import numpy as np
import pylab as pl
from scipy.interpolate import spline

#To get all query conditions
handle_query_condition()
        
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'RESOURCE')):
    #Declare a Sketchpad as the first graph
    all_physical_read_list = []
    x_ticks = []
    x_ticks_lables = []
    y_physical_reads_max = 0
    legend_title_physical = []
    phy_mil = False
    for id_1 in range(len(DBs)):
        #Get data from the target database to assemble a data frame that will be used for the following graphs
        dataframe_hour = %sql select char(date(collected)) as date, hour(collected) as hours, sum(total_cpu_usec_delta) / 1000000.0 as cpu_sec, sum(logical_reads_delta) as logical_reads, sum(PHYSICAL_READS_DELTA) as physical_reads, sum(total_act_time)/(sum(act_aborted_total)+sum(act_completed_total)) as avg_activity_time_msec,sum(act_completed_total_delta) as activities from ibm_dsm_views.throughput_all where dbconn_id = '{DBs[id_1]}' and collected >= '{START_TIME}' and collected < '{END_TIME}' group by date(collected), hour(collected) order by date, hours
        if dataframe_hour.empty:
            if resource_df_empty == False:
                print 'For resource(Physical Reads): The query result is empty for the database connection ' + DBs[id_1] + ', please check your query parameters.\n'
        else:
            legend_title_physical.append(DBs[id_1])
            #Get the hour data as x-axis from the data frame
            x_hour = dataframe_hour['HOURS']
            y_physical_reads = dataframe_hour['PHYSICAL_READS']
            y_physical_reads_list = list(y_physical_reads)
            if (y_physical_reads_max < y_physical_reads.max()):
                y_physical_reads_max = y_physical_reads.max()

            x_hour_list = list(x_hour) #here x_hour is a python list
            date_all = list(dataframe_hour['DATE'].values)

            ori_datetime_str = get_original_datatime_str(date_all, x_hour_list)

            '''
            When the data queried is not equal to the requirement. Filling missing data 
            into the list x_hour_list,date_all and y_ticks.
            '''
            if (len(x_hour_list) < INTERVAL):
                y_physical_reads_list = []
                #Transfer datatime data into time tuple for getting its' timestamp
                min_dt_str = START_TIME[0:13] + ':00:00' 
                tm_tuple = time.strptime(min_dt_str, '%Y-%m-%d %H:00:00')
                min_timestamp = time.mktime(tm_tuple)
            
                date_ref_str = END_TIME[0:13] + ':00:00'
                tmp_tuple = time.strptime(date_ref_str, '%Y-%m-%d %H:00:00')
                max_timestamp = time.mktime(tmp_tuple)
                '''
                Get the difference between the max_timestamp and min_timestamp 
                which will be used for gaining all date and hour including the missing
                '''
                hour_diff = int((max_timestamp - min_timestamp) / 3600)
                #clear date_all
                date_all = []
                #clear x_hour_list
                x_hour_list = []
                #clear y_ticks
                
                #Reassign three var above
                for id_2 in range(hour_diff):
                    tmp_st = min_timestamp + id_2 * 3600
                    #Trans timestamp to datetime string
                    tmp_datetime = datetime.datetime.fromtimestamp(tmp_st)
                    tmp_datetime_str = tmp_datetime.strftime("%Y-%m-%d %H:%M:%S")
                    tmp_date_str = tmp_datetime_str[0:10]
                    tmp_hour_str = tmp_datetime_str[11:13]
                    date_all.append(tmp_date_str)
                    x_hour_list.append(tmp_hour_str)
                    
                    '''
                    Below code is for handling y-axis's data
                    If there is no data at this hour,0.0 will be filled into
                    '''
                    if tmp_datetime_str in ori_datetime_str:
                        tmp_index = ori_datetime_str[tmp_datetime_str]
                        y_physical_reads_list.append(list(y_physical_reads)[tmp_index])
                    else:
                        y_physical_reads_list.append(0)
            all_physical_read_list.append(y_physical_reads_list)
           
            #previous date string value as a reference data
            pre_date_str = str(date_all[0])
            
            if len(x_ticks) == 0:
                for id_3 in range(len(date_all)):
                    #Get the data for the x-axis ticks
                    x_ticks.append(float('%.1f' % id_3))

                    '''
                    Get the data for the lable of x-axis
                    If the label existed in the list x_ticks_lables,
                    put hour_str into x_ticks_lables 
                    otherwise,put date_lables into x_ticks_lables
                    '''
                    hour_str = str(x_hour_list[id_3])
                    date_str = str(date_all[id_3])
                    x_lables = date_str + ' ' + hour_str
                    if (id_3 == 0):
                        x_ticks_lables.append(x_lables)
                    else:#id_3 > 0
                        if (pre_date_str == date_str):
                            x_ticks_lables.append(hour_str)
                        else:
                            pre_date_str = date_str
                            x_ticks_lables.append(x_lables)
                        
    if len(all_physical_read_list) > 0:
        #Declare a Sketchpad as the first graph
        fig = pl.figure()
        ax_phy = fig.add_subplot(111)
        #Declare an ax container as the first drawing paper
        x_lable= 'Hours'
        data_size = len(x_ticks)
        if(data_size <= 20):
            fig.set_size_inches(12,6)
        elif(data_size <= 40):
            fig.set_size_inches(16,6)
        elif(data_size <= 60):
            fig.set_size_inches(18,7)
        elif(data_size <=100):
            fig.set_size_inches(22,7)

        pl.xlabel(x_lable, fontsize = 12)
        #Set title for the second graph
        figure_title='Physical Reads by Hour\n'
        pl.title(figure_title, fontsize = 14, fontweight = 'bold')
        pl.xticks(x_ticks, x_ticks_lables, rotation = 90)
        #To set grid line style according to your requirement
        pl.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
        
        #Set y-axis label
        if y_physical_reads.max() > 1000000:
            phy_mil = True
            for id_4 in range(len(all_physical_read_list)):
                for id_4_0 in range(len(all_physical_read_list[id_4])):
                    all_physical_read_list[id_4][id_4_0] = all_physical_read_list[id_4][id_4_0]/1000000.0
        
        xnew_hour = []
        ynew_physical_reads = []
        for id_5 in range(len(all_physical_read_list)):
            #To mark the data point 
            for id_6 in range(len(all_physical_read_list[id_5])):
                if(all_physical_read_list[id_5][id_6] == 0.0):#If no data,drawing a empty circle
                    pl.scatter(x_ticks[id_6],all_physical_read_list[id_5][id_6], c = '', marker = 'o', edgecolors = 'r', s = 50)
                else:
                    pl.scatter(x_ticks[id_6], all_physical_read_list[id_5][id_6], c = '#2c628b') 
                    pl.text(x_ticks[id_6], all_physical_read_list[id_5][id_6], '%.1f' % all_physical_read_list[id_5][id_6], fontsize = 9)
        
            if(len(all_physical_read_list[id_5]) >= 3):
                ##In order to smooth the line chart,handle the data further##
                #Expand each x axis data 20 times
                xnew_hour = np.linspace(np.asarray(x_ticks).min(), np.asarray(x_ticks).max(), np.asarray(x_ticks).size*20) 
                #Handle the data of new y axis data
                ynew_physical_reads = spline(np.asarray(x_ticks), all_physical_read_list[id_5], xnew_hour)
                ynew_physical_reads_list = list(ynew_physical_reads)
                #No negative value for y-axis
                for id_7 in range(len(ynew_physical_reads_list)):
                    if (ynew_physical_reads_list[id_7] < 0.0):
                        ynew_physical_reads_list[id_7] = 0.0
                ynew_physical_reads = np.asarray(ynew_physical_reads_list)        
                #Fill the gragh according to your requirement
                #pl.fill_between(xnew_hour, ynew_physical_reads, where=(xnew_hour.min()<xnew_hour) & (xnew_hour<xnew_hour.max()), color = '#2c628b', alpha = 0.09)
                #Draw curve graph
                pl.plot(xnew_hour, ynew_physical_reads)
               
        if (phy_mil == True):
            pl.ylabel(u'PHYSICAL_READS(mil times)', fontsize = 12)
        else:
            pl.ylabel(u'PHYSICAL_READS(times)',fontsize=12)
        #pl.legend(labels = ['a', 'b'], loc = 'best')
        #Set the legends for the both graphs
        box_phy = ax_phy.get_position()
        ax_phy.set_position([box_phy.x0, box_phy.y0 + box_phy.height * 0.1, box_phy.width, box_phy.height * 0.9])
        ax_phy.legend(legend_title_physical, fontsize = 11, loc = 'upper center', bbox_to_anchor=(0.5,1.08), fancybox = True, shadow = True, ncol = len(legend_title_physical))
        pl.show()

In [ ]:
#import the required lib
import numpy as np
import pylab as pl
from scipy.interpolate import spline

#To get all query conditions
handle_query_condition()
        
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'RESOURCE')):
    #Declare a Sketchpad as the first graph
    all_cpu_list = []
    x_ticks = []
    x_ticks_lables = []
    legend_title_cpu = []
    for id_1 in range(len(DBs)):
        #Get data from the target database to assemble a data frame that will be used for the following graphs
        dataframe_hour = %sql select char(date(collected)) as date, hour(collected) as hours, sum(total_cpu_usec_delta) / 1000000.0 as cpu_sec, sum(logical_reads_delta) as logical_reads, sum(PHYSICAL_READS_DELTA) as physical_reads, sum(total_act_time)/(sum(act_aborted_total)+sum(act_completed_total)) as avg_activity_time_msec,sum(act_completed_total_delta) as activities from ibm_dsm_views.throughput_all where dbconn_id = '{DBs[id_1]}' and collected >= '{START_TIME}' and collected < '{END_TIME}' group by date(collected), hour(collected) order by date, hours
        if dataframe_hour.empty:
            if resource_df_empty == False:
                print 'For resource(CPU_SEC): The query result is empty for the database connection ' + DBs[id_1] + ', please check your query parameters.\n'
        else:
            legend_title_cpu.append(DBs[id_1])
            #Get the hour data as x-axis from the data frame
            x_hour = dataframe_hour['HOURS']
            y_cpu = dataframe_hour['CPU_SEC']
            y_cpu_list = list(y_cpu)

            x_hour_list = list(x_hour) #here x_hour is a python list
            date_all = list(dataframe_hour['DATE'].values)

            ori_datetime_str = get_original_datatime_str(date_all, x_hour_list)

            '''
            When the data queried is not equal to the requirement. Filling missing data 
            into the list x_hour_list,date_all and y_ticks.
            '''
            if (len(x_hour_list) < INTERVAL):
                y_cpu_list = []
                #Transfer datatime data into time tuple for getting its' timestamp
                min_dt_str = START_TIME[0:13] + ':00:00' 
                tm_tuple = time.strptime(min_dt_str, '%Y-%m-%d %H:00:00')
                min_timestamp = time.mktime(tm_tuple)
            
                date_ref_str = END_TIME[0:13] + ':00:00'
                tmp_tuple = time.strptime(date_ref_str, '%Y-%m-%d %H:00:00')
                max_timestamp = time.mktime(tmp_tuple)
                '''
                Get the difference between the max_timestamp and min_timestamp 
                which will be used for gaining all date and hour including the missing
                '''
                hour_diff = int((max_timestamp - min_timestamp) / 3600)
                #clear date_all
                date_all = []
                #clear x_hour_list
                x_hour_list = []
                #clear y_ticks
                
                #Reassign three var above
                for id_2 in range(hour_diff):
                    tmp_st = min_timestamp + id_2 * 3600
                    #Trans timestamp to datetime string
                    tmp_datetime = datetime.datetime.fromtimestamp(tmp_st)
                    tmp_datetime_str = tmp_datetime.strftime("%Y-%m-%d %H:%M:%S")
                    tmp_date_str = tmp_datetime_str[0:10]
                    tmp_hour_str = tmp_datetime_str[11:13]
                    date_all.append(tmp_date_str)
                    x_hour_list.append(tmp_hour_str)
                    
                    '''
                    Below code is for handling y-axis's data
                    If there is no data at this hour,0.0 will be filled into
                    '''
                    if tmp_datetime_str in ori_datetime_str:
                        tmp_index = ori_datetime_str[tmp_datetime_str]
                        y_cpu_list.append(list(y_cpu)[tmp_index])
                    else:
                        y_cpu_list.append(0)
            all_cpu_list.append(y_cpu_list)
           
            #previous date string value as a reference data
            pre_date_str = str(date_all[0])
            
            if len(x_ticks) == 0:
                for id_3 in range(len(date_all)):
                    #Get the data for the x-axis ticks
                    x_ticks.append(float('%.1f' % id_3))

                    '''
                    Get the data for the lable of x-axis
                    If the label existed in the list x_ticks_lables,
                    put hour_str into x_ticks_lables 
                    otherwise,put date_lables into x_ticks_lables
                    '''
                    hour_str = str(x_hour_list[id_3])
                    date_str = str(date_all[id_3])
                    x_lables = date_str + ' ' + hour_str
                    if (id_3 == 0):
                        x_ticks_lables.append(x_lables)
                    else:#id_3 > 0
                        if (pre_date_str == date_str):
                            x_ticks_lables.append(hour_str)
                        else:
                            pre_date_str = date_str
                            x_ticks_lables.append(x_lables)
                        
    if len(all_cpu_list) > 0:
        #Declare a Sketchpad as the first graph
        fig = pl.figure()
        ax_cpu = fig.add_subplot(111)
        #Declare an ax container as the first drawing paper
        x_lable= 'Hours'
        data_size = len(x_ticks)
        if(data_size <= 20):
            fig.set_size_inches(12,6)
        elif(data_size <= 40):
            fig.set_size_inches(16,6)
        elif(data_size <= 60):
            fig.set_size_inches(18,7)
        elif(data_size <=100):
            fig.set_size_inches(22,7)

        pl.xlabel(x_lable, fontsize = 12)
        #Set title for the second graph
        figure_title='CPU Usage by Hour\n'
        pl.title(figure_title, fontsize = 14, fontweight = 'bold')
        pl.xticks(x_ticks, x_ticks_lables, rotation = 90)
        #To set grid line style according to your requirement
        pl.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
        
        #Set y-axis label
        xnew_hour = []
        ynew_cpu = []
        for id_5 in range(len(all_cpu_list)):
            #To mark the data point 
            for id_6 in range(len(all_cpu_list[id_5])):
                if(all_cpu_list[id_5][id_6] == 0.0):#If no data,drawing a empty circle
                    pl.scatter(x_ticks[id_6],all_cpu_list[id_5][id_6], c = '', marker = 'o', edgecolors = 'r', s = 50)
                else:
                    pl.scatter(x_ticks[id_6], all_cpu_list[id_5][id_6], c = '#2c628b') 
                    pl.text(x_ticks[id_6], all_cpu_list[id_5][id_6], '%.1f' % all_cpu_list[id_5][id_6], fontsize = 9)
        
            if(len(all_cpu_list[id_5]) >= 3):
                ##In order to smooth the line chart,handle the data further##
                #Expand each x axis data 20 times
                xnew_hour = np.linspace(np.asarray(x_ticks).min(), np.asarray(x_ticks).max(), np.asarray(x_ticks).size*20) 
                #Handle the data of new y axis data
                ynew_cpu = spline(np.asarray(x_ticks), all_cpu_list[id_5], xnew_hour)
                ynew_cpu_list = list(ynew_cpu)
                #No negative value for y-axis
                for id_7 in range(len(ynew_cpu_list)):
                    if (ynew_cpu_list[id_7] < 0.0):
                        ynew_cpu_list[id_7] = 0.0
                ynew_cpu = np.asarray(ynew_cpu_list)        
                #Fill the gragh according to your requirement
                #pl.fill_between(xnew_hour, ynew_cpu, where=(xnew_hour.min()<xnew_hour) & (xnew_hour<xnew_hour.max()), color = '#2c628b', alpha = 0.09)
                #Draw curve graph
                pl.plot(xnew_hour, ynew_cpu)
               
        pl.ylabel(u'CPU_SEC(s)',fontsize=12)
        #Set the legends for the both graphs
        box_cpu = ax_cpu.get_position()
        ax_cpu.set_position([box_cpu.x0, box_cpu.y0 + box_cpu.height * 0.1, box_cpu.width, box_cpu.height * 0.9])
        ax_cpu.legend(legend_title_cpu, fontsize = 11, loc = 'upper center', bbox_to_anchor=(0.5,1.08), fancybox = True, shadow = True, ncol = len(legend_title_cpu))
        pl.show()

In [ ]:
#import the required lib
import numpy as np
import pylab as pl
from scipy.interpolate import spline

#To get all query conditions
handle_query_condition()
        
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'RESOURCE')):
    #Declare a Sketchpad as the first graph
    all_avg_act_list = []
    x_ticks = []
    x_ticks_lables = []
    legend_title_avg_act = []
    for id_1 in range(len(DBs)):
        #Get data from the target database to assemble a data frame that will be used for the following graphs
        dataframe_hour = %sql select char(date(collected)) as date, hour(collected) as hours, sum(total_cpu_usec_delta) / 1000000.0 as cpu_sec, sum(logical_reads_delta) as logical_reads, sum(PHYSICAL_READS_DELTA) as physical_reads, sum(total_act_time)/(sum(act_aborted_total)+sum(act_completed_total)) as avg_activity_time_msec,sum(act_completed_total_delta) as activities from ibm_dsm_views.throughput_all where dbconn_id = '{DBs[id_1]}' and collected >= '{START_TIME}' and collected < '{END_TIME}' group by date(collected), hour(collected) order by date, hours
        if dataframe_hour.empty:
            if resource_df_empty == False:
                print 'For resource(AVG_ACTIVITY_TIME_MSEC): The query result is empty for the database connection ' + DBs[id_1] + ', please check your query parameters.\n'
        else:
            legend_title_avg_act.append(DBs[id_1])
            #Get the hour data as x-axis from the data frame
            x_hour = dataframe_hour['HOURS']
            y_avg_act = dataframe_hour['AVG_ACTIVITY_TIME_MSEC']
            y_avg_act_list = list(y_avg_act)

            x_hour_list = list(x_hour) #here x_hour is a python list
            date_all = list(dataframe_hour['DATE'].values)

            ori_datetime_str = get_original_datatime_str(date_all, x_hour_list)

            '''
            When the data queried is not equal to the requirement. Filling missing data 
            into the list x_hour_list,date_all and y_ticks.
            '''
            if (len(x_hour_list) < INTERVAL):
                y_avg_act_list = []
                #Transfer datatime data into time tuple for getting its' timestamp
                min_dt_str = START_TIME[0:13] + ':00:00' 
                tm_tuple = time.strptime(min_dt_str, '%Y-%m-%d %H:00:00')
                min_timestamp = time.mktime(tm_tuple)
            
                date_ref_str = END_TIME[0:13] + ':00:00'
                tmp_tuple = time.strptime(date_ref_str, '%Y-%m-%d %H:00:00')
                max_timestamp = time.mktime(tmp_tuple)
                '''
                Get the difference between the max_timestamp and min_timestamp 
                which will be used for gaining all date and hour including the missing
                '''
                hour_diff = int((max_timestamp - min_timestamp) / 3600)
                #clear date_all
                date_all = []
                #clear x_hour_list
                x_hour_list = []
                #clear y_ticks
                
                #Reassign three var above
                for id_2 in range(hour_diff):
                    tmp_st = min_timestamp + id_2 * 3600
                    #Trans timestamp to datetime string
                    tmp_datetime = datetime.datetime.fromtimestamp(tmp_st)
                    tmp_datetime_str = tmp_datetime.strftime("%Y-%m-%d %H:%M:%S")
                    tmp_date_str = tmp_datetime_str[0:10]
                    tmp_hour_str = tmp_datetime_str[11:13]
                    date_all.append(tmp_date_str)
                    x_hour_list.append(tmp_hour_str)
                    
                    '''
                    Below code is for handling y-axis's data
                    If there is no data at this hour,0.0 will be filled into
                    '''
                    if tmp_datetime_str in ori_datetime_str:
                        tmp_index = ori_datetime_str[tmp_datetime_str]
                        y_avg_act_list.append(list(y_avg_act)[tmp_index])
                    else:
                        y_avg_act_list.append(0)
            all_avg_act_list.append(y_avg_act_list)
           
            #previous date string value as a reference data
            pre_date_str = str(date_all[0])
            
            if len(x_ticks) == 0:
                for id_3 in range(len(date_all)):
                    #Get the data for the x-axis ticks
                    x_ticks.append(float('%.1f' % id_3))

                    '''
                    Get the data for the lable of x-axis
                    If the label existed in the list x_ticks_lables,
                    put hour_str into x_ticks_lables 
                    otherwise,put date_lables into x_ticks_lables
                    '''
                    hour_str = str(x_hour_list[id_3])
                    date_str = str(date_all[id_3])
                    x_lables = date_str + ' ' + hour_str
                    if (id_3 == 0):
                        x_ticks_lables.append(x_lables)
                    else:#id_3 > 0
                        if (pre_date_str == date_str):
                            x_ticks_lables.append(hour_str)
                        else:
                            pre_date_str = date_str
                            x_ticks_lables.append(x_lables)
                        
    if len(all_avg_act_list) > 0:
        #Declare a Sketchpad as the first graph
        fig = pl.figure()
        ax_avg_act = fig.add_subplot(111)
        #Declare an ax container as the first drawing paper
        x_lable= 'Hours'
        data_size = len(x_ticks)
        if(data_size <= 20):
            fig.set_size_inches(12,6)
        elif(data_size <= 40):
            fig.set_size_inches(16,6)
        elif(data_size <= 60):
            fig.set_size_inches(18,7)
        elif(data_size <=100):
            fig.set_size_inches(22,7)

        pl.xlabel(x_lable, fontsize = 12)
        #Set title for the second graph
        figure_title='Average Activity Time by Hour\n'
        pl.title(figure_title, fontsize = 14, fontweight = 'bold')
        pl.xticks(x_ticks, x_ticks_lables, rotation = 90)
        #To set grid line style according to your requirement
        pl.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
        
        #Set y-axis label
        xnew_hour = None
        ynew_avg_act = None
        for id_5 in range(len(all_avg_act_list)):
            #To mark the data point 
            for id_6 in range(len(all_avg_act_list[id_5])):
                if(all_avg_act_list[id_5][id_6] == 0.0):#If no data,drawing a empty circle
                    pl.scatter(x_ticks[id_6],all_avg_act_list[id_5][id_6], c = '', marker = 'o', edgecolors = 'r', s = 50)
                else:
                    pl.scatter(x_ticks[id_6], all_avg_act_list[id_5][id_6], c = '#2c628b') 
                    pl.text(x_ticks[id_6], all_avg_act_list[id_5][id_6], '%.0f' % all_avg_act_list[id_5][id_6], fontsize = 9)
        
            if(len(all_avg_act_list[id_5]) >= 3):
                ##In order to smooth the line chart,handle the data further##
                #Expand each x axis data 20 times
                xnew_hour = np.linspace(np.asarray(x_ticks).min(), np.asarray(x_ticks).max(), np.asarray(x_ticks).size*20) 
                #Handle the data of new y axis data
                ynew_avg_act = spline(np.asarray(x_ticks), all_avg_act_list[id_5], xnew_hour)
                ynew_avg_act_list = list(ynew_avg_act)
                #No negative value for y-axis
                for id_7 in range(len(ynew_avg_act_list)):
                    if (ynew_avg_act_list[id_7] < 0.0):
                        ynew_avg_act_list[id_7] = 0.0
                ynew_avg_act = np.asarray(ynew_avg_act_list)        
                #Fill the gragh according to your requirement
                #pl.fill_between(xnew_hour, ynew_avg_act, where=(xnew_hour.min()<xnew_hour) & (xnew_hour<xnew_hour.max()), color = '#2c628b', alpha = 0.09)
                #Draw curve graph
                pl.plot(xnew_hour, ynew_avg_act)
               
        pl.ylabel(u'AVG_ACTIVITY_TIME_MSEC', fontsize = 12)
        #Set the legends for the both graphs
        box_avg_act = ax_avg_act.get_position()
        ax_avg_act.set_position([box_avg_act.x0, box_avg_act.y0 + box_avg_act.height * 0.1, box_avg_act.width, box_avg_act.height * 0.9])
        ax_avg_act.legend(legend_title_avg_act, fontsize = 11, loc = 'upper center', bbox_to_anchor=(0.5,1.08), fancybox = True, shadow = True, ncol = len(legend_title_avg_act))
        pl.show()

In [ ]:
import numpy as np
import pandas as pd
import os

handle_query_condition()

#Query condition exists some error
if len(GENERAL_ERROR) > 0 and os.path.exists("cpu.csv"):
    !rm cpu.csv
 
def get_all_times(list_all, list_distinct):
    try:
        distinct_times = {}
        for idx in range(len(list_distinct)):
            cnt = 0
            for idx_1 in range(len(list_all)):
                if list_distinct[idx] == list_all[idx_1]:
                    cnt += 1
            distinct_times[str(list_distinct[idx])] = cnt
        return distinct_times
    except Exception as e:
        print str(e)
        
def getmaxtime(conn_times):
    try:
        maxtime = 1
        all_times = conn_times.values()
        for idx in range(len(all_times)):
            if maxtime < all_times[idx]:
                maxtime = all_times[idx]
        return maxtime
    except Exception as e:
        print str(e)
        
def get_sort_circle(mean_cpu_usage):
    try:
        circle_size = {}
        dbname_keys = []
        cpu_values = []
        tmp_dict = {}
        ##Get no duplicates values
        for k,v in mean_cpu_usage.items():
            dbname_keys.append(k)
            cpu_values.append(v)
        cpu_values = sorted(list(set(cpu_values)))
        
        ##Form a new dictionary to store above key and rank value
        for i in range(len(cpu_values)):
            tmp_dict[cpu_values[i]] = i + 1
        
        #Generate a new dictionary using new rank
        for key in mean_cpu_usage:
            for key1 in tmp_dict:
                if mean_cpu_usage[key] == key1:
                    circle_size[key] = tmp_dict[key1]
        return circle_size
    
    except Exception as e:
        print str(e)
    
def get_mean_rank(all_conn_names,distinct_conn_names, all_ranks):
    try:
        y_data = []
        y_data_dict = {}
        score_list = {1:13, 2:8, 3:5, 4:3, 5:2}
        for i in range(len(distinct_conn_names)):
            tmp_cnt = 0
            tmp_score = 0
            for j in range(len(all_conn_names)):
                if str(distinct_conn_names[i]) == str(all_conn_names[j]):
                    tmp_cnt += 1
                    tmp_score += score_list[all_ranks[j]]
            float_rank = float('%.1f' % (tmp_score * 1.0 / tmp_cnt))
            y_data.append(float_rank)
            y_data_dict[str(distinct_conn_names[i])] = float_rank
        
        float_sorted_rank = list(reversed(sorted(list(set(y_data)))))
        int_rank_dict = {}
        ##Form a new dictionary to store above key and rank value
        for k in range(len(float_sorted_rank)):
            int_rank_dict[float_sorted_rank[k]] = k + 1
        
        for f in range(len(y_data)):
            for key in int_rank_dict:
                if(y_data[f]) == key:
                    y_data[f] = 6 - int_rank_dict[key]
                    break
        return y_data

    except Exception as e:
        print str(e)
    
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'CPU')):
    dataframe_cpu = %sql with temp as(select row_number()over(partition by week(collected) order by sum(total_cpu_usec_delta)/1000000.0 desc) as row_id, month(collected) as month, week(collected) as week, dbconn_id as conn_name, decimal(sum(total_cpu_usec_delta)/1000000.0,17,2) as cpu_sec from ibm_dsm_views.throughput_all group by month(collected), week(collected), dbconn_id order by month, week, row_id) select month, week, conn_name, cpu_sec, row_id as rank from temp where row_id <= 5
    if os.path.exists("cpu.csv"):
        !rm cpu.csv
    if dataframe_cpu.empty:
        print 'For CPU: The query result is empty, please check your query parameters.\n'
    else:
        #To save log data for generating table
        dataframe_cpu.to_csv("cpu.csv", index_label = "INDEX")
        #For the x-labels
        x_ticks_labels = list(set(list(dataframe_cpu['WEEK'].values)))
        x_ticks = list(range(1, len(x_ticks_labels) + 1))
        x_ticks_labels = list(set(list(dataframe_cpu['WEEK'].values)))
        #For the y-labels
        y_ticks = list(range(1, 6))
        y_ticks_labels = list(reversed(y_ticks))
        #Get all the connection names from the data frame
        all_conn_names = list(dataframe_cpu['CONN_NAME'].values)
        #Remove duplicates
        distinct_conn_names = list(set(all_conn_names))
        #For the size of scatter
        conn_times = get_all_times(all_conn_names, distinct_conn_names)
        #Get weeks
        all_weeks = list(dataframe_cpu['WEEK'].values)
        all_ranks = list(dataframe_cpu['RANK'].values)
        
        fig = pl.figure()
        fig.set_size_inches(24,6)
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        #60 colors
        all_colors = ['#988FD0', '#E04C32', '#368AB6', '#777777', '#FEBE67', '#66CDAA', '#0000CD', '#BA55D3', '#9370DB', '#3CB371', '#7B68EE', '#00FA9A', '#48D1CC', '#C71585', '#191970', '#F5FFFA', '#FFE4E1', '#FFE4B5', '#FFDEAD', '#000080', '#FDF5E6', '#808000', '#6B8E23', '#FFA500', '#FF4500', '#DA70D6', '#EEE8AA', '#98FB98', '#AFEEEE', '#DB7093', '#FFEFD5', '#FFDAB9', '#CD853F', '#FFC0CB', '#DDA0DD', '#B0E0E6', '#800080', '#FF0000', '#BC8F8F', '#4169E1', '#8B4513', '#FA8072', '#FAA460', '#2E8B57', '#FFF5EE', '#A0522D', '#C0C0C0', '#87CEEB', '#6A5ACD', '#708090', '#FFFAFA', '#00FF7F', '#4682B4', '#D2B48C', '#008080', '#D8BFD8', '#FF6347', '#40E0D0', '#EE82EE', '#F5DEB3']
        ax1.set_title('Weekly Top 5 CPU Usage\n',fontsize = 14, fontweight = 'bold')  
        ax1.set_xlabel('Weeks', fontsize = 12)
        ax1.set_ylabel('Rank', fontsize = 12)  
        ax1.set_xticks(x_ticks)
        ax1.set_xticklabels(x_ticks_labels)
        ax1.set_yticks(y_ticks)
        ax1.set_yticklabels(y_ticks_labels)
        ax1.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
        
        mean_cpu_usage = {}
        #Get all the connection names from the data frame
        all_cpu_usage = list(dataframe_cpu['CPU_SEC'].values)
        
        for id_0 in range(len(distinct_conn_names)):
            x_data = []
            y_data = []
            cnt = 0
            sum_cpu_usage = 0.0
            for id_1 in range(len(all_conn_names)):
                if(distinct_conn_names[id_0] == all_conn_names[id_1]):
                    cnt += 1
                    sum_cpu_usage += all_cpu_usage[id_1]
                    x_data.append(x_ticks_labels.index(all_weeks[id_1]) + 1)
                    y_data.append(6 - all_ranks[id_1])
                
            mean_cpu_usage[str(distinct_conn_names[id_0])] = float('%.1f' % (sum_cpu_usage / cnt))
            ax1.scatter(x_data, y_data, c = all_colors[id_0], marker = 'o', edgecolors = 'r', s = 200 * conn_times[distinct_conn_names[id_0]])
            ax1.text(x_data[0], y_data[0], '%s' % distinct_conn_names[id_0], fontsize = 9, verticalalignment = 'top', horizontalalignment = 'right')
            ax1.plot(x_data, y_data, linestyle = '--', color = all_colors[id_0], lw = 1.0)
        
        #Below is the second graph
        ax2.set_title('Top CPU Usage Map\n',fontsize = 14, fontweight = 'bold')  
        ax2.set_xlabel('Frequency', fontsize = 12)
        ax2.set_ylabel('Rank', fontsize = 12)
        max_times = getmaxtime(conn_times)
        x_ticks = list(range(1, max_times + 1))
        x_data = conn_times.values()
        y_data = get_mean_rank(all_conn_names, distinct_conn_names, all_ranks)
        circle_size = get_sort_circle(mean_cpu_usage)
        ax2.set_xticks(x_ticks)
        ax2.set_xticklabels(x_ticks)
        ax2.set_yticks(y_ticks)
        ax2.set_yticklabels(y_ticks_labels)
        ax2.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
        for ax2_idx in range(len(distinct_conn_names)):
            ax2.scatter(x_data[ax2_idx], y_data[ax2_idx], c = all_colors[ax2_idx], marker = 'o', edgecolors = 'r', s = 200 * circle_size[distinct_conn_names[ax2_idx]])
            ax2.text(x_data[ax2_idx], y_data[ax2_idx], '%s' % distinct_conn_names[ax2_idx], fontsize = 9, verticalalignment = 'top', horizontalalignment = 'left')
        pl.show()

In [ ]:
import numpy as np
import pandas as pd
import os
import math

handle_query_condition()

#Query condition exists some error
if len(GENERAL_ERROR) > 0 and os.path.exists("memory.csv"):
    !rm memory.csv

def get_all_times(list_all, list_distinct):
    try:
        distinct_times = {}
        for idx in range(len(list_distinct)):
            cnt = 0
            for idx_1 in range(len(list_all)):
                if list_distinct[idx] == list_all[idx_1]:
                    cnt += 1
            distinct_times[str(list_distinct[idx])] = cnt
        return distinct_times
    except Exception as e:
        print str(e)
        
def getmaxtime(conn_times):
    try:
        maxtime = 1
        all_times = conn_times.values()
        for idx in range(len(all_times)):
            if maxtime < all_times[idx]:
                maxtime = all_times[idx]
        return maxtime
    except Exception as e:
        print str(e)
        
def get_sort_circle(mean_memory_usage):
    try:
        circle_size = {}
        dbname_keys = []
        memory_values = []
        tmp_dict = {}
        ##Get no duplicates values
        for k,v in mean_memory_usage.items():
            dbname_keys.append(k)
            memory_values.append(v)
        memory_values = sorted(list(set(memory_values)))
        
        ##Form a new dictionary to store above key and rank value
        for i in range(len(memory_values)):
            tmp_dict[memory_values[i]] = i + 1
        
        #Generate a new dictionary using new rank
        for key in mean_memory_usage:
            for key1 in tmp_dict:
                if mean_memory_usage[key] == key1:
                    circle_size[key] = tmp_dict[key1]
        return circle_size
    
    except Exception as e:
        print str(e)
    
def get_mean_rank(all_conn_names,distinct_conn_names, all_ranks):
    try:
        y_data = []
        y_data_dict = {}
        score_list = {1:13, 2:8, 3:5, 4:3, 5:2}
        for i in range(len(distinct_conn_names)):
            tmp_cnt = 0
            tmp_score = 0
            for j in range(len(all_conn_names)):
                if str(distinct_conn_names[i]) == str(all_conn_names[j]):
                    tmp_cnt += 1
                    tmp_score += score_list[all_ranks[j]]
            float_rank = float('%.1f' % (tmp_score * 1.0 / tmp_cnt))
            y_data.append(float_rank)
            y_data_dict[str(distinct_conn_names[i])] = float_rank
        
        float_sorted_rank = list(reversed(sorted(list(set(y_data)))))
        int_rank_dict = {}
        ##Form a new dictionary to store above key and rank value
        for k in range(len(float_sorted_rank)):
            int_rank_dict[float_sorted_rank[k]] = k + 1
        
        for f in range(len(y_data)):
            for key in int_rank_dict:
                if(y_data[f]) == key:
                    y_data[f] = 6 - int_rank_dict[key]
                    break
        return y_data

    except Exception as e:
        print str(e)
    
if(ISOK_ALL_PARA and (REPORT_TYPE == 'ALL' or REPORT_TYPE == 'MEMORY')):
    dataframe_memory = %sql with temp as(select row_number()over(partition by week(collected) order by max(memory_pool_used_gb) desc) as row_id, month(collected) as month, week(collected) as week, dbconn_id as conn_name, dec(max(memory_pool_used_gb),17,2) as memory_usage_gb from IBM_DSM_VIEWS.MEM_DB_TOTAL_USED group by month(collected), week(collected), dbconn_id order by month, week, row_id) select month, week, conn_name, memory_usage_gb, row_id as rank from temp where row_id < 5
    
    if os.path.exists("memory.csv"):
        !rm memory.csv
    if dataframe_memory.empty:
        print 'For Memory: The query result is empty, please check your query parameters.\n'
    else:
        #To save log data for generating table
        dataframe_memory.to_csv("memory.csv", index_label = "INDEX")
        
        #For the x-labels
        x_ticks_labels = list(set(list(dataframe_memory['WEEK'].values)))
        x_ticks = list(range(1, len(x_ticks_labels) + 1))
        x_ticks_labels = list(set(list(dataframe_memory['WEEK'].values)))
        #For the y-labels
        y_ticks = list(range(1, 6))
        y_ticks_labels = list(reversed(y_ticks))
        #Get all the connection names from the data frame
        all_conn_names = list(dataframe_memory['CONN_NAME'].values)
        #Remove duplicates
        distinct_conn_names = list(set(all_conn_names))
        #For the size of scatter
        conn_times = get_all_times(all_conn_names, distinct_conn_names)
        #Get weeks
        all_weeks = list(dataframe_memory['WEEK'].values)
        all_ranks = list(dataframe_memory['RANK'].values)
        fig = pl.figure()
        fig.set_size_inches(24,6)
        ax1 = fig.add_subplot(121)
        ax2 = fig.add_subplot(122)
        #60 colors
        all_colors = ['#988FD0', '#E04C32', '#368AB6', '#777777', '#FEBE67', '#66CDAA', '#0000CD', '#BA55D3', '#9370DB', '#3CB371', '#7B68EE', '#00FA9A', '#48D1CC', '#C71585', '#191970', '#F5FFFA', '#FFE4E1', '#FFE4B5', '#FFDEAD', '#000080', '#FDF5E6', '#808000', '#6B8E23', '#FFA500', '#FF4500', '#DA70D6', '#EEE8AA', '#98FB98', '#AFEEEE', '#DB7093', '#FFEFD5', '#FFDAB9', '#CD853F', '#FFC0CB', '#DDA0DD', '#B0E0E6', '#800080', '#FF0000', '#BC8F8F', '#4169E1', '#8B4513', '#FA8072', '#FAA460', '#2E8B57', '#FFF5EE', '#A0522D', '#C0C0C0', '#87CEEB', '#6A5ACD', '#708090', '#FFFAFA', '#00FF7F', '#4682B4', '#D2B48C', '#008080', '#D8BFD8', '#FF6347', '#40E0D0', '#EE82EE', '#F5DEB3']
        ax1.set_title('Weekly Top 5 Memory Usage\n',fontsize = 14, fontweight = 'bold')  
        ax1.set_xlabel('Weeks', fontsize = 12)
        ax1.set_ylabel('Rank', fontsize = 12)  
        ax1.set_xticks(x_ticks)
        ax1.set_xticklabels(x_ticks_labels)
        ax1.set_yticks(y_ticks)
        ax1.set_yticklabels(y_ticks_labels)
        ax1.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
        
        mean_memory_usage = {}
        #Get all the connection names from the data frame
        all_memory_usage = list(dataframe_memory['MEMORY_USAGE_GB'].values)

        for id_0 in range(len(distinct_conn_names)):
            x_data = []
            y_data = []
            cnt = 0
            sum_memory_usage = 0.0
            for id_1 in range(len(all_conn_names)):
                if(distinct_conn_names[id_0] == all_conn_names[id_1]):
                    cnt += 1
                    sum_memory_usage += all_memory_usage[id_1]
                    x_data.append(x_ticks_labels.index(all_weeks[id_1]) + 1)
                    y_data.append(6 - all_ranks[id_1])
            mean_memory_usage[str(distinct_conn_names[id_0])] = float('%.1f' % (sum_memory_usage / cnt))
            ax1.scatter(x_data, y_data, c = all_colors[id_0], marker = 'o', edgecolors = 'r', s = 200 * conn_times[distinct_conn_names[id_0]])
            ax1.text(x_data[0], y_data[0], '%s' % distinct_conn_names[id_0], fontsize = 9, verticalalignment = 'top', horizontalalignment = 'right')
            ax1.plot(x_data, y_data, linestyle = '--', color = all_colors[id_0], lw = 1.0)
        
        #Below is the second graph
        ax2.set_title('Top Memory Usage Map\n',fontsize = 14, fontweight = 'bold')  
        ax2.set_xlabel('Frequency', fontsize = 12)
        ax2.set_ylabel('Rank', fontsize = 12)
        max_times = getmaxtime(conn_times)
        x_ticks = list(range(1, max_times + 1))
        x_data = conn_times.values()
        y_data = get_mean_rank(all_conn_names, distinct_conn_names, all_ranks)
        circle_size = get_sort_circle(mean_memory_usage)
        ax2.set_xticks(x_ticks)
        ax2.set_xticklabels(x_ticks)
        ax2.set_yticks(y_ticks)
        ax2.set_yticklabels(y_ticks_labels)
        ax2.grid(True, ls = '--', color = '#2c628b', alpha = 0.05)
        for ax2_idx in range(len(distinct_conn_names)):
            ax2.scatter(x_data[ax2_idx], y_data[ax2_idx], c = all_colors[ax2_idx], marker = 'o', edgecolors = 'r', s = 200 * circle_size[distinct_conn_names[ax2_idx]])
            ax2.text(x_data[ax2_idx], y_data[ax2_idx], '%s' % distinct_conn_names[ax2_idx], fontsize = 9, verticalalignment = 'top', horizontalalignment = 'left')
        pl.show()       